# AACGM-v2 library crib

This notebook demonstrates how to use the python library [aacgmv2](https://github.com/aburrell/aacgmv2) by Angeline Burrell. This library is a Python wrapper for the AACGM-v2 C library.

The examples in this notebook are similar to the examples included in the IDL version of the [AACGM-v2 v2.7 release](https://superdarn.thayer.dartmouth.edu/aacgm.html). A similar crib is also included in IDL SPEDAS (see the IDL SPEDAS file external/aacgm_v2/aacgmidl_v2_crib.pro). 

Below, we show the IDL results (as of March 2025) in order to compare the python results to the IDL results. In all cases they should be the same or very close.

In [15]:
# In python, the user needs to manually install the aacgmv2 library:   
# pip install aacgmv2
#
# If the library is not installed, the following will print a message.

try:
    import aacgmv2
    print("aacgmv2 has been imported")  
except ImportError:
    print("aacgmv2 is not installed")   


# Import other required libraries.
import datetime as dt
import numpy as np

aacgmv2 has been imported


## 1. Test single input G2A.

The python function aacgmv2.convert_latlon() is similar to the IDL function cnvcoord_v2.

<sub>

Expected output (using IDL, with date 1997-06-25):

| MLAT       | MLON        | R          |
|------------|-------------|------------|
| 44.594873  | -167.37238  | 1.0165480  |

</sub>

In [16]:
# Define the date.
dtime = dt.datetime(1997, 6, 25, 0, 0, 0)

# Convert between geomagnetic coordinates and AACGM coordinates.
lat0 = 50.
lon0 = 120.
alt  = 111.
p = np.array(aacgmv2.convert_latlon(lat0, lon0, alt, dtime))

# Print results.
print("Actual output (python):")
print(p)

Actual output (python):
[  44.59487304 -167.37238414    1.01654805]


## 2. Test field line tracing, single input, G2A Trace.

If we want to use line tracing instead of IGRF coefficients, we need to set method_code="trace".

<sub>

Expected output (using IDL, with date 1997-06-25):

| MLAT       | MLON        | R          |
|------------|-------------|------------|
| 44.591538  | -167.37413  | 1.0165480  |

</sub>

In [17]:
# Define the date.
dtime = dt.datetime(1997, 6, 25, 0, 0, 0)

# Convert between geomagnetic coordinates and AACGM coordinates using trace.
lat0 = 50.
lon0 = 120.
alt  = 111.
p = np.array(aacgmv2.convert_latlon(lat0, lon0, alt, dtime, method_code="trace"))

# Print results.
print("Actual output (python):")
print(p)

Actual output (python):
[  44.59153809 -167.37412764    1.01654805]


## 3. Test vector input.

When we have vector inputs instead of scalars, we must use the function aacgmv2.convert_latlon_arr().

<sub>

Expected output (using IDL):

|   GLAT         |   GLON         |   HEIGHT    |   
|----------------|----------------|-------------|
|   45.207731    |  -166.00805    |   1.0165480 |
|   51.964642    |   123.86140    |   1.0380817 |
|  -44.966218    |  -34.130621    |   1.0932993 |
|  -88.619636    |   87.301848    |   1.0450458 |
|   38.043788    |   89.310874    |   1.2983149 |
|   19.391025    |   122.80231    |   1.0241234 |
|   21.336952    |   17.480107    |   1.0527631 |

</sub>

In [18]:
# Define the date.
dtime = dt.datetime(2014, 1, 22, 0, 0, 0)

# Convert between geomagnetic coordinates and AACGM coordinates using vector input.
input = np.array([[50.,120.,111.], [55.,50.,250.], [-50.,-120.,600.], [-75.,120.,300.], 
    [33,15,1900], [23,50,150], [11,-60,330]])

p = np.array(aacgmv2.convert_latlon_arr(input[:,0], input[:,1], input[:,2], dtime))

# Print results
print('Actual output (python):')
# Print the transposed array, to make it easier to compare with the expected output from IDL.
print(p.T)

Actual output (python):
[[  45.20773144 -166.00805111    1.01654805]
 [  51.9646415   123.86139828    1.0380817 ]
 [ -44.96621805  -34.13062066    1.0932993 ]
 [ -88.61963647   87.30184773    1.04504578]
 [  38.04378847   89.31087357    1.29831494]
 [  19.39102518  122.80230958    1.02412339]
 [  21.33695241   17.48010654    1.05276313]]


## 4. Test inverse transformation.

For Geographic (geodetic) to AACGM-v2 we use method_code="G2A".

For AACGM-v2 to geographic (geodetic) we use method_code="A2G".


<sub>

Expected output (using IDL):

|   GLAT       |   GLON       |   HEIGHT      |   Description                                  |
|-------------:|-------------:|--------------:|------------------------------------------------|
|  50.000000   |  12.000000   |  450.000000  |  original geographic (lat lon height)          |
|  47.262737   |  88.118328   |  1.069756    |  AACGM coordinates (coefficients)              |
|  47.267521   |  88.113706   |  1.069756    |  AACGM coordinates (field-line tracing)        |
|  49.962937   |  11.990158   |  449.986370  |  Inverse xform using coefficients              |
|  49.999998   |  12.000000   |  450.000013  |  Inverse xform using field-line tracing        |

Note that the inverse transformation is less accurate than the
 forward transformation when using coefficients. Using field line
 tracing for the inverse transformation restores accuracy.
</sub>


In [19]:
lat0 = 50.
lon0 = 12.
alt  = 450.
RE = 6371.2
dtime = dt.datetime(2014, 1, 22, 0, 0, 0)
p = aacgmv2.convert_latlon(lat0,lon0, alt, dtime, method_code="G2A")         # G2A using coefficients
s = aacgmv2.convert_latlon(lat0,lon0, alt, dtime, method_code="G2A|trace")   # G2A using field-line tracing
hp = RE*(p[2]-1.)
hs = RE*(s[2]-1.)
q = aacgmv2.convert_latlon(p[0], p[1], hp, dtime, method_code="A2G")        # inverse xform using coeffs
r = aacgmv2.convert_latlon(s[0], s[1], hs, dtime, method_code="A2G|trace")  # inv using tracing

# Print results
print('Actual output (python):')
print("Original geographic (lat lon height)")
print(lat0, lon0, alt)  
print("")
print("AACGM coordinates using coefficients")
print(p)
print("AACGM coordinates using field-line tracing")
print(s)
print("")
print("Inverse xform using coefficients")
print(q)
print("Inverse xform using field-line tracing")
print(r) 

Actual output (python):
Original geographic (lat lon height)
50.0 12.0 450.0

AACGM coordinates using coefficients
(47.262737200263764, 88.11832800392624, 1.0697559668683327)
AACGM coordinates using field-line tracing
(47.2675213539154, 88.11370644927747, 1.0697559668683327)

Inverse xform using coefficients
(49.962936919802324, 11.99015760144865, 449.98635683386465)
Inverse xform using field-line tracing
(49.99999874698483, 11.999999989835468, 449.99999953884685)


## 5. Magnetic local time (MLT)  

For MLT, use the function aacgmv2.convert_mlt(arr, dtime, m2a=False).

The parameter m2a=True converts MLT to AACGM-v2 longitude, while m2a=False (the default) converts magnetic longitude to MLT.

<sub>

Expected output (using IDL):

| GLAT       | GLON        | HEIGHT      | MLAT       | MLON        | R         | MLT       |
|------------|-------------|-------------|------------|-------------|-----------|-----------|
| 77.000000  | -88.000000  | 300.000000  | 85.518716  | -25.478611  | 1.044990  | 1.412911  |
| 77.000000  | -88.000000  | 300.000000  | 85.515863  | -25.478538  | 1.044990  | 1.412915  |

</sub>

In [20]:
dtime = dt.datetime(2003, 5, 17, 7, 53, 16)
lat = 77.0 
lon = -88.0 
hgt = 300.0
p = [lat, lon, hgt]  #   input in geographic coordinates
p2 = aacgmv2.convert_latlon(lat, lon, hgt, dtime)   # compute AACGM-v2 coordinates of point
m2 = aacgmv2.convert_mlt(p2[1], dtime)   # compute MLT using AACGM-v2 longitude
p3 = aacgmv2.convert_latlon(lat, lon, hgt, dtime, method_code="trace")    # compute AACGM-v2 coordinates of point
m3 = aacgmv2.convert_mlt(p3[1], dtime)              # compute MLT using AACGM-v2 longitude

# Print results
print('Actual output (python):')
fmt = "{:10.6f} {:11.6f} {:11.6f} {:10.6f} {:11.6f} {:8.6f}  {:8.6f}"
print(' GLAT       GLON        HEIGHT      MLAT       MLON       R         MLT')
print(fmt.format(lat, lon, hgt, p2[0], p2[1], p2[2], m2[0]))
print(fmt.format(lat, lon, hgt, p3[0], p3[1], p3[2], m3[0]))

Actual output (python):
 GLAT       GLON        HEIGHT      MLAT       MLON       R         MLT
 77.000000  -88.000000  300.000000  85.518716  -25.478611 1.044990  1.412911
 77.000000  -88.000000  300.000000  85.515863  -25.478538 1.044990  1.412915


## 6. MLT for vector input

This example shows how to use vector input.


<sub>

Expected output (using IDL):

| GLAT       | GLON       | HEIGHT     | MLAT       | MLON       | R         | MLT      |
|------------|------------|------------|------------|------------|-----------|----------|
| 45.000000  | 0.000000   | 150.000000 | 40.281106  | 76.676596  | 1.022961  | 8.223258 |
| 45.000000  | 1.000000   | 150.000000 | 40.241518  | 77.499622  | 1.022961  | 8.278126 |
| 45.000000  | 2.000000   | 150.000000 | 40.207406  | 78.326105  | 1.022961  | 8.333225 |
| 45.000000  | 3.000000   | 150.000000 | 40.178556  | 79.156156  | 1.022961  | 8.388562 |
| 45.000000  | 4.000000   | 150.000000 | 40.154752  | 79.989873  | 1.022961  | 8.444143 |
| 45.000000  | 5.000000   | 150.000000 | 40.135781  | 80.827344  | 1.022961  | 8.499974 |
| 45.000000  | 6.000000   | 150.000000 | 40.121428  | 81.668645  | 1.022961  | 8.556061 |
| 45.000000  | 7.000000   | 150.000000 | 40.111480  | 82.513848  | 1.022961  | 8.612408 |
| 45.000000  | 8.000000   | 150.000000 | 40.105725  | 83.363012  | 1.022961  | 8.669019 |
| 45.000000  | 9.000000   | 150.000000 | 40.103954  | 84.216191  | 1.022961  | 8.725897 |
| 45.000000  | 10.000000  | 150.000000 | 40.105959  | 85.073434  | 1.022961  | 8.783047 |
| 45.000000  | 11.000000  | 150.000000 | 40.111537  | 85.934780  | 1.022961  | 8.840470 |
| 45.000000  | 12.000000  | 150.000000 | 40.120485  | 86.800267  | 1.022961  | 8.898169 |
| 45.000000  | 13.000000  | 150.000000 | 40.132603  | 87.669924  | 1.022961  | 8.956146 |
| 45.000000  | 14.000000  | 150.000000 | 40.147694  | 88.543778  | 1.022961  | 9.014403 |
| 45.000000  | 15.000000  | 150.000000 | 40.165563  | 89.421850  | 1.022961  | 9.072941 |
| 45.000000  | 16.000000  | 150.000000 | 40.186016  | 90.304158  | 1.022961  | 9.131762 |
| 45.000000  | 17.000000  | 150.000000 | 40.208861  | 91.190715  | 1.022961  | 9.190866 |
| 45.000000  | 18.000000  | 150.000000 | 40.233909  | 92.081530  | 1.022961  | 9.250253 |
| 45.000000  | 19.000000  | 150.000000 | 40.260970  | 92.976609  | 1.022961  | 9.309925 |

</sub>

In [21]:
dtime = dt.datetime(2003, 5, 17, 7, 53, 16)
npts = 20
lons = np.arange(npts, dtype=float)   
lats = 45.0 + np.zeros(npts, dtype=float)   
hgts = 150.0 + np.zeros(npts, dtype=float)  

p4 = aacgmv2.convert_latlon_arr(lats, lons, hgts, dtime)
m4 = aacgmv2.convert_mlt(p4[1], dtime)   # compute MLT using AACGM-v2 longitude

# Print results in a table format
fmt = "{:10.6f} {:11.6f} {:11.6f} {:10.6f} {:11.6f} {:8.6f}  {:8.6f}"

# Print the header
print(f" GLAT       GLON        HEIGHT      MLAT       MLON       R         MLT") 

# Print the table
for k in range(npts):
    print(f"{fmt.format(lats[k], lons[k], hgts[k], p4[0][k], p4[1][k], p4[2][k], m4[k])}")

 GLAT       GLON        HEIGHT      MLAT       MLON       R         MLT
 45.000000    0.000000  150.000000  40.281106   76.676596 1.022961  8.223258
 45.000000    1.000000  150.000000  40.241518   77.499622 1.022961  8.278126
 45.000000    2.000000  150.000000  40.207406   78.326105 1.022961  8.333225
 45.000000    3.000000  150.000000  40.178556   79.156156 1.022961  8.388562
 45.000000    4.000000  150.000000  40.154752   79.989873 1.022961  8.444143
 45.000000    5.000000  150.000000  40.135781   80.827344 1.022961  8.499974
 45.000000    6.000000  150.000000  40.121428   81.668645 1.022961  8.556061
 45.000000    7.000000  150.000000  40.111480   82.513848 1.022961  8.612408
 45.000000    8.000000  150.000000  40.105725   83.363012 1.022961  8.669019
 45.000000    9.000000  150.000000  40.103954   84.216191 1.022961  8.725897
 45.000000   10.000000  150.000000  40.105959   85.073434 1.022961  8.783047
 45.000000   11.000000  150.000000  40.111537   85.934780 1.022961  8.840470
 45.